In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")


# class MyQueryEndEventHandler(BaseEventHandler):
#     @classmethod
#     def class_name(cls) -> str:
#         """Class name."""
#         return "MyQueryEndEventHandler"

#     def handle(self, event) -> None:
#         """Logic for handling event."""
#         if isinstance(event, QueryEndEvent):
#             print(self.class_name())
#             with open("log.txt", "a") as f:
#                 f.write(str(event))
#                 f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher("app")

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents, dispatcher=dispatcher)

In [ ]:
retriever = index.as_retriever()

In [ ]:
retriever.dispatcher

Dispatcher(name='app', event_handlers=[MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent=Dispatcher(name='root', event_handlers=[NullEventHandler()], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent=None, propagate=False), propagate=True)

In [ ]:
query_engine = index.as_query_engine()

retriever dispatcher: name='app' event_handlers=[MyEventHandler()] span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]) parent=Dispatcher(name='root', event_handlers=[NullEventHandler()], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent=None, propagate=False) propagate=True


In [ ]:
query_engine.dispatcher

Dispatcher(name='app', event_handlers=[MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent=Dispatcher(name='root', event_handlers=[NullEventHandler()], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent=None, propagate=False), propagate=True)

In [ ]:
query_engine._response_synthesizer.dispatcher

Dispatcher(name='app', event_handlers=[MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent=Dispatcher(name='root', event_handlers=[NullEventHandler()], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent=None, propagate=False), propagate=True)

In [ ]:
query_result = query_engine.query("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
QueryEndEvent


In [ ]:
query_engine.dispatcher.span_handler.completed_spans

[SimpleSpan(id_='retrieve-6cc1f7dd-dd63-47d5-88f4-93401ac4a806', parent_id='query-75e7b287-231b-44d0-bffd-02564dacf867', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 66248), end_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249150), duration=0.182902),
 SimpleSpan(id_='get_response-897267ed-36fe-4890-a312-94ccccbecde5', parent_id='synthesize-a2c476ce-266c-4378-ade7-174fc2a9d218', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249292), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 275620), duration=1.026328),
 SimpleSpan(id_='synthesize-a2c476ce-266c-4378-ade7-174fc2a9d218', parent_id='query-75e7b287-231b-44d0-bffd-02564dacf867', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249212), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 275685), duration=1.026473),
 SimpleSpan(id_='query-75e7b287-231b-44d0-bffd-02564dacf867', parent_id=None, start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 63844), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 276340), du

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
QueryEndEvent


In [ ]:
query_engine.dispatcher.span_handler.completed_spans

[SimpleSpan(id_='retrieve-6cc1f7dd-dd63-47d5-88f4-93401ac4a806', parent_id='query-75e7b287-231b-44d0-bffd-02564dacf867', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 66248), end_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249150), duration=0.182902),
 SimpleSpan(id_='get_response-897267ed-36fe-4890-a312-94ccccbecde5', parent_id='synthesize-a2c476ce-266c-4378-ade7-174fc2a9d218', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249292), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 275620), duration=1.026328),
 SimpleSpan(id_='synthesize-a2c476ce-266c-4378-ade7-174fc2a9d218', parent_id='query-75e7b287-231b-44d0-bffd-02564dacf867', start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 249212), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 275685), duration=1.026473),
 SimpleSpan(id_='query-75e7b287-231b-44d0-bffd-02564dacf867', parent_id=None, start_time=datetime.datetime(2024, 3, 4, 14, 23, 12, 63844), end_time=datetime.datetime(2024, 3, 4, 14, 23, 13, 276340), du